# Single-process Tests

In [1]:
!pip install *.whl

Processing ./cucim-0.19.0-py3-none-manylinux2014_x86_64.whl
cuclara-image is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [2]:
!pip install pandas altair altair_viewer

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
from contextlib import ContextDecorator
from time import perf_counter

class Timer(ContextDecorator):
    def __init__(self, data, lib, start_location, patch_size):
        self.data = data
        self.lib = lib
        self.start_location = start_location
        self.patch_size = patch_size
    def __enter__(self):
        self.start = perf_counter()
    def __exit__(self, exc_type, exc, exc_tb):
        self.end = perf_counter()
        data.append([self.lib, self.start_location, self.patch_size, self.end - self.start])

In [4]:
from cucim import CuImage
from openslide import OpenSlide

import pandas as pd


patch_sizes = (256, 512, 1024) #(32, 64, 128, 256, 512, 1024, 2048, 4096)


start_locs = (0, 1, 256, 512, 1024, 4096) #(4096 + 256, 4096 + 256 * 2, 4096 + 256 * 3, 4096 + 256 * 4, 8192, 8192 + 1024, 8192 + 1024 + 1)
repeat = 30

for start_loc in start_locs:
    data = []
    for patch_size in patch_sizes:

        # cuCIM
        for i in range(repeat):
            with Timer(data, 'cuCIM(reload)', f'({start_loc},{start_loc})', patch_size):
                img = CuImage("input/image.tif")
                region = img.read_region([start_loc, start_loc], [patch_size, patch_size], 0)

        img = CuImage("input/image.tif")
        for i in range(repeat):
            with Timer(data, 'cuCIM(reuse)', f'({start_loc},{start_loc})', patch_size):
                region = img.read_region([start_loc, start_loc], [patch_size, patch_size],0)

        # OpenSlide
        for i in range(repeat):
            with Timer(data, 'openslide(reload)', f'({start_loc},{start_loc})', patch_size):
                img2 = OpenSlide("input/image.tif")
                region2 = img2.read_region([start_loc, start_loc], 0, [patch_size, patch_size])

        img2 = OpenSlide("input/image.tif")
        for i in range(repeat):
            with Timer(data, 'openslide(reuse)', f'({start_loc},{start_loc})', patch_size):
                region2 = img2.read_region([start_loc, start_loc], 0, [patch_size, patch_size])

    df = pd.DataFrame(data=data, columns=['Library', 'Start position', 'Patch Size', 'Time'] )
#     print(start_loc)
#     print("    cuCIM: {:.5f}".format(df[repeat*0:][:repeat]['Time'].mean()))
#     print("    OpenSlide   : {:.5f}".format(df[repeat*2:][:repeat]['Time'].mean()))
#     print()
#     print("    Gain        : {:.2f}".format(df[repeat*2:][:repeat]['Time'].mean() / df[repeat*0:][:repeat]['Time'].mean()))

#     print("{},{:.5f},{:.5f},{:.2f}".format(start_loc, df[repeat*1:][:repeat]['Time'].mean(), df[repeat*3:][:repeat]['Time'].mean(), df[repeat*3:][:repeat]['Time'].mean() / df[repeat*1:][:repeat]['Time'].mean()))

#     print("    OpenSlide   : {:.5f}".format())
#     print()
#     print("    Gain        : {:.2f}".format(df[repeat*2:][:repeat]['Time'].mean() / df[repeat*0:][:repeat]['Time'].mean()))
            


In [5]:
import pandas as pd
df = pd.DataFrame(data=data, columns=['Library', 'Start position', 'Patch Size', 'Time'] )
#df.to_csv(f'data_{start_loc}.csv')
#df.groupby(['Patch Size', 'Library']).mean()['Time'].to_csv(f"time_{start_loc}.csv")
df

,Library,Start position,Patch Size,Time
0,cuCIM(reload),"(4096,4096)",256,0.000573
1,cuCIM(reload),"(4096,4096)",256,0.000489
2,cuCIM(reload),"(4096,4096)",256,0.000434
3,cuCIM(reload),"(4096,4096)",256,0.000432
4,cuCIM(reload),"(4096,4096)",256,0.000429
...,...,...,...,...
355,openslide(reuse),"(4096,4096)",1024,0.025841
356,openslide(reuse),"(4096,4096)",1024,0.022839
357,openslide(reuse),"(4096,4096)",1024,0.025384
358,openslide(reuse),"(4096,4096)",1024,0.022736


In [7]:
import altair as alt

bars = alt.Chart(df).mark_bar().encode(
    x='average(Time):Q',
    y='Library:O',
    color='Library:N',
    row='Patch Size:N',
    text=alt.Text('average(Time):Q', format='.3f')
).properties(
    width=600,
    height=100
)
bars

alt.Chart(...)

In [9]:
import time

print("cuCIM")
for i in range(10):
    s = time.time()
    img = CuImage("input/image.tif")
    region = img.read_region([10000,10000], [512,512], 0)
    e = time.time()
    print("End", e - s)
print()

print("cuCIM (without reopening)")
for i in range(10):
    s = time.time()
    region = img.read_region([10000,10000], [512,512], 0)
    e = time.time()
    print("End", e - s)
print()

from openslide import OpenSlide

print("OpenSlide")
for i in range(10):
    s = time.time()
    img2 = OpenSlide("input/image.tif")
    region2 = img2.read_region([10000,10000], 0, [512,512])
    e = time.time()
    print("End", e-s)

    img2 = OpenSlide("input/image.tif")
print()

print("OpenSlide (without reopening)")
for i in range(10):
    s = time.time()
    region2 = img2.read_region([10000,10000], 0, [512,512])
    e = time.time()
    print("End", e-s)
print()


cuCIM
End 0.003916740417480469
End 0.0035552978515625
End 0.003542661666870117
End 0.0035364627838134766
End 0.0035610198974609375
End 0.003505706787109375
End 0.0036716461181640625
End 0.003631114959716797
End 0.0035119056701660156
End 0.003532886505126953

cuCIM (without reopening)
End 0.0032813549041748047
End 0.0032656192779541016
End 0.003276824951171875
End 0.003265380859375
End 0.0032677650451660156
End 0.0032622814178466797
End 0.0032775402069091797
End 0.0032546520233154297
End 0.0032753944396972656
End 0.003261566162109375

OpenSlide
End 0.011467456817626953
End 0.010767698287963867
End 0.010530948638916016
End 0.010488033294677734
End 0.01053619384765625
End 0.010490179061889648
End 0.010544538497924805
End 0.010465621948242188
End 0.010503292083740234
End 0.010487079620361328

OpenSlide (without reopening)
End 0.009581804275512695
End 0.005589723587036133
End 0.005550384521484375
End 0.0054779052734375
End 0.0055086612701416016
End 0.005495309829711914
End 0.005474805831909